Test queries on chicago_bikes database 

In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
# import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirm list of databases
print(mongo.list_database_names())

['admin', 'chicago_bikes', 'classDB', 'config', 'fruits_db', 'gardenDB', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# Assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# Review collections in database
print(db.list_collection_names())

['divvy_rides_by_season', 'withLatLong', 'withoutStationName', 'Top10EndStations', 'weather_daily', 'Top10StartStations', 'withStationName', 'divvy_ridedata_merged', 'divvy_ridedata', 'RouteDistance', 'Top10Routes', 'divvy_rides_by_month']


In [6]:
# Assign weather data to a variable 
weather_daily = db['weather_daily']

In [7]:
# Count the number of documents in weather data 
print(weather_daily.count_documents({}))

365


In [8]:
# Review a document in the weather_daily collection 
print(db.weather_daily.find_one())

{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41}


In [9]:
# Convert precipitation field value to inches (currently in mm)
# Conversion factor: 1 mm = 0.03937 inches
mm_to_inches = 0.03937

In [10]:
# Iterate through documents and update values
for document in weather_daily.find():
    size_mm = document.get("precipitation")
    if size_mm is not None:
        size_inches = size_mm * mm_to_inches
        # Update the document with the new size value in inches
        weather_daily.update_one({"_id": document["_id"]}, {"$set": {"prcp_inches": size_inches}})

In [11]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41, 'prcp_inches': 0.5436997}


In [12]:
# Drop 'size_inches' field as it has been replaced with 'prcp_inches' and values match 
# Specify the field you want to drop
field_to_drop = "size_inches"

In [13]:
# Update documents to unset the specified field
weather_daily.update_many({}, {"$unset": {field_to_drop: 1}})

In [14]:
# Review a document in the weather_daily collection to confirm drop
print(db.weather_daily.find_one())

{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41, 'prcp_inches': 0.5436997}


In [15]:
# Define the threshold value for significant precipitation comparison 
threshold = 0.1

In [16]:
# Update documents to add the new field "sig_prcp"
# Add a new field 'sig_prcp' and populate 'yes' values if 'prcp_inches' is greather than or equal .1 inches
weather_daily.update_many(
    {"prcp_inches": {"$gte": threshold}},
    {"$set": {"sig_prcp": "yes"}}
)

weather_daily.update_many(
    {"prcp_inches": {"$lt": threshold}},
    {"$set": {"sig_prcp": "no"}}
)

In [17]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41, 'prcp_inches': 0.5436997, 'sig_prcp': 'yes'}


In [18]:
# Update documents to add the new field "avg_temp"
weather_daily.update_many(
    {},
    [
        {
            "$set": {
                "avg_temp": {
                    "$avg": ["$morning_temp", "$afternoon_temp", "$evening_temp"]
                }
            }
        }
    ]
)

In [19]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('64e434c9b6c5bdee4f41aa4b'), 'date': '2022-01-02', 'cloud_cover': 90, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41, 'prcp_inches': 0.5436997, 'sig_prcp': 'yes', 'avg_temp': 24.836666666666662}


In [20]:
# Close the MongoDB connection
client.close()

NameError: name 'client' is not defined